In [ ]:
!pip install -U coremltools==3.0b6 tensorflow==1.13.1 keras==2.2.4


In [ ]:
!pip install -U turicreate

     |████████████████████████████████| 91.4MB 66kB/s 
     |████████████████████████████████| 327kB 43.4MB/s 
     |████████████████████████████████| 3.4MB 40.9MB/s 
  Created wheel for resampy: filename=resampy-0.2.1-cp36-none-any.whl size=320850 sha256=c3f3477d9c492e6e5421c40569969235b5ede460bbb7900ee55a4f9c1dc6852a
  Stored in directory: /root/.cache/pip/wheels/ff/4f/ed/2e6c676c23efe5394bb40ade50662e90eb46e29b48324c5f9b
Successfully built resampy
  Found existing installation: resampy 0.2.2
    Uninstalling resampy-0.2.2:
      Successfully uninstalled resampy-0.2.2


In [ ]:
import numpy as np
import pickle
import os
import sys
import glob
import pandas as pd
import turicreate as tc
from pathlib import Path
from keras.models import Sequential
from keras.layers import Dense,Flatten,Dropout,LSTM,Bidirectional
from keras.utils import to_categorical
from sklearn.metrics import mean_squared_error, mean_absolute_error, accuracy_score

Using TensorFlow backend.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')
DATA_PATH = "/content/drive/My Drive/Colab Notebooks/HW/Prev/Data/Session/"

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
#START TURI
DATA_PATH = "/content/drive/My Drive/Colab Notebooks/HW/Prev/Data/Session/"

In [ ]:
def read_pkl(filename):
  with open(filename, 'rb') as file:
    data = pickle.load(file, encoding='latin1')
    return data

def write_pkl(filename, dict):
  with open(filename, 'wb') as file:
    pickle.dump(dict, file)

def get_label_for_time(time,annots):
  for index, row in annots.iterrows():
    tm = int(row['time'])
    stm = int(row['start_time'])
    etm = int(row['end_time'])
    cd = int(row['code'])
    if cd == 101 or cd == 102 or cd == 103:
      if time >= tm and time <= etm:
        return 1
  return 0

def populate_df(sessionId,df,sensor,annots,containsHW): #returns 2D list
  idx = 0
  for index, row in sensor.iterrows():
    lbl = 0
    if containsHW == True:
      lbl = get_label_for_time(row["time"],annots)
    df.loc[idx] = [sessionId,row["userAccX"],row["userAccY"],row["userAccZ"],row["gravX"],row["gravY"],row["gravZ"],row["rotatX"],row["rotatY"],row["rotatZ"],lbl]
    idx += 1

  return df

In [ ]:
#Skip this- all done and saved
all_raw_files = glob.glob(DATA_PATH + "*.pkl")
cols = ['SessionId','AccX','AccY','AccZ','GravX','GravY','GravZ','RotX','RotY','RotZ','Activity']
l_df = pd.DataFrame(columns=cols)
r_df = pd.DataFrame(columns=cols)

for filename in all_raw_files:
  fn_parts = str(Path(filename).resolve().stem).split('_')
  sessionId = int(fn_parts[0])
  print("...Inside session", str(Path(filename).resolve().stem))
  data = read_pkl(filename)
  lsensor = data['left_sensors']['imu']
  rsensor = data['right_sensors']['imu']
  annots = data['annotations']
  containsHW = True
  if fn_parts[2] == 'nhw':
    containsHW = False
  l_df = populate_df(sessionId,l_df,lsensor,annots,containsHW)
  print("left sensor done")
  r_df = populate_df(sessionId,r_df,rsensor,annots,containsHW)
  print("right sensor done")
  l_df.to_csv(DATA_PATH + str(sessionId) +'_left.csv', index= False)
  r_df.to_csv(DATA_PATH + str(sessionId) +'_right.csv', index= False)

print("Over")

...Inside session 1_ab_1
left sensor done
right sensor done
...Inside session 2_ab_2
left sensor done
right sensor done
...Inside session 3_ab_3
left sensor done
right sensor done
...Inside session 4_az_1
left sensor done
right sensor done
...Inside session 5_az_2
left sensor done
right sensor done
...Inside session 6_em_1
left sensor done
right sensor done
...Inside session 7_em_2
left sensor done
right sensor done
...Inside session 8_li_2
left sensor done
right sensor done
...Inside session 9_li_3
left sensor done
right sensor done
...Inside session 10_li_4
left sensor done
right sensor done
...Inside session 11_li_5
left sensor done
right sensor done
...Inside session 12_si_1
left sensor done
right sensor done
...Inside session 13_si_2
left sensor done
right sensor done
...Inside session 14_si_3
left sensor done
right sensor done
...Inside session 16_si_nhw_2
left sensor done
right sensor done
...Inside session 15_li_1
left sensor done
right sensor done
...Inside session 17_si_nhw_1

In [ ]:
all_csv_files = glob.glob(DATA_PATH + "/*.csv")
l_dfs = []
r_dfs = []

for filename in all_csv_files:
    df = pd.read_csv(filename, index_col=None, header=0)
    print("Loaded File:", filename, df.shape)
    if ("left" in filename):
      l_dfs.append(df)
    elif ("right" in filename):
      r_dfs.append(df)

#merge data from all files
l_df = pd.concat(l_dfs, axis=0, ignore_index=True)
r_df = pd.concat(r_dfs, axis=0, ignore_index=True)

Loaded File: /content/drive/My Drive/Colab Notebooks/HW/Prev/Data/Session/1_left.csv (44070, 11)
Loaded File: /content/drive/My Drive/Colab Notebooks/HW/Prev/Data/Session/1_right.csv (44163, 11)
Loaded File: /content/drive/My Drive/Colab Notebooks/HW/Prev/Data/Session/2_left.csv (44070, 11)
Loaded File: /content/drive/My Drive/Colab Notebooks/HW/Prev/Data/Session/2_right.csv (44163, 11)
Loaded File: /content/drive/My Drive/Colab Notebooks/HW/Prev/Data/Session/3_left.csv (44070, 11)
Loaded File: /content/drive/My Drive/Colab Notebooks/HW/Prev/Data/Session/3_right.csv (44163, 11)
Loaded File: /content/drive/My Drive/Colab Notebooks/HW/Prev/Data/Session/4_left.csv (44070, 11)
Loaded File: /content/drive/My Drive/Colab Notebooks/HW/Prev/Data/Session/4_right.csv (44163, 11)
Loaded File: /content/drive/My Drive/Colab Notebooks/HW/Prev/Data/Session/5_left.csv (44070, 11)
Loaded File: /content/drive/My Drive/Colab Notebooks/HW/Prev/Data/Session/5_right.csv (44163, 11)
Loaded File: /content/dri

In [ ]:
print(r_df.shape, l_df.shape)

(1214144, 11) (1206676, 11)


In [ ]:
l_sf = tc.SFrame(data=l_df) #tc.SFrame.read_csv(DATA_PATH + 'left_DF.csv')
l_sf

SessionId,AccX,AccY,AccZ,GravX
1.0,-0.01457,0.015309999999999999,-0.00562,0.39760999999999996
1.0,-0.01925,-0.00365,-0.007659999999999999,0.39834
1.0,-0.00446,0.00946,-0.010790000000000001,0.39919
1.0,0.00168,0.01207,-0.01,0.3999
1.0,0.0035399999999999997,0.009609999999999999,-0.02445,0.39998
1.0,0.18175999999999998,0.06651,0.00758,0.40058
1.0,-0.00625,0.02513,-0.02546,0.40005
1.0,0.00881,0.00139,-0.006409999999999999,0.39958
1.0,0.00645,0.00569,0.00295,0.39903
1.0,-0.1157,-0.04278,-0.00954,0.47236999999999996


In [ ]:
l_sf.groupby('Activity', [tc.aggregate.COUNT]).sort("Count", ascending = False)

Activity,Count
0.0,1131000
1.0,75676


In [ ]:
l_sf['Activity'] = l_sf['Activity'].astype(int)
l_sf['SessionId'] = l_sf['SessionId'].astype(int)

In [ ]:
train, test = tc.activity_classifier.util.random_split_by_session(l_sf, session_id='SessionId', fraction=0.8)

In [ ]:
model = tc.activity_classifier.create(train, session_id='SessionId', target='Activity', prediction_window=150)

Pre-processing 650147 samples...

Using sequences of size 3000 for model creation.

Processed a total of 58 sessions.

Using CPU to create model

+--------------+----------------+--------------+

...

| 6            | 0.948759       | 5.97s        |

| 10           | 0.969081       | 8.46s        |

+--------------+----------------+--------------+

Training complete

Total Time Spent: 11.8455

In [ ]:
metrics = model.evaluate(test)
print(metrics)

{'accuracy': 0.9778951321494478, 'auc': 0.9590312450342949, 'confusion_matrix': Columns:
	class	int
	predicted_class	int
	count	int

Rows: 3

Data:
+-------+-----------------+--------+
| class | predicted_class | count  |
+-------+-----------------+--------+
|   0   |        0        | 543927 |
|   1   |        0        | 12302  |
|   1   |        1        |  300   |
+-------+-----------------+--------+
[3 rows x 3 columns]
, 'f1_score': 0.046504417919702376, 'log_loss': 0.150768320311513, 'precision': 1.0, 'recall': 0.02380574511982225, 'roc_curve': Columns:
	threshold	float
	fpr	float
	tpr	float
	p	int
	n	int

Rows: 100001

Data:
+-----------+-----+-----+-------+--------+
| threshold | fpr | tpr |   p   |   n    |
+-----------+-----+-----+-------+--------+
|    0.0    | 1.0 | 1.0 | 12602 | 543927 |
|   1e-05   | 1.0 | 1.0 | 12602 | 543927 |
|   2e-05   | 1.0 | 1.0 | 12602 | 543927 |
|   3e-05   | 1.0 | 1.0 | 12602 | 543927 |
|   4e-05   | 1.0 | 1.0 | 12602 | 543927 |
|   5e-05   | 1.

In [ ]:
print(metrics['accuracy'])

0.9783443450386233


In [ ]:
model.export_coreml(DATA_PATH + 'ActivityClassifier.mlmodel')

In [ ]:
!turicreate --version

/bin/bash: turicreate: command not found


In [ ]:
##END TURI

In [ ]:
#functions model
def get_RNN_model():
  biLSTM_model = Sequential()
  biLSTM_model.add(Bidirectional(LSTM(100, return_sequences=True, stateful=False), input_shape=(30, 9)))
  biLSTM_model.add(Dense(1, activation='sigmoid'))
  biLSTM_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
  return biLSTM_model

def train_model_unseen_id(data, labels, unseenid):
  trainx, trainy = [],[]
  for a in subj_dict.items():
    subjid = a[1]
    if subjid != unseenid:
      trainx.extend(data[subjid])
      trainy.extend(labels[subjid])

  trainx, trainy = np.array(trainx), np.array(trainy)
  trainy = trainy[:, :, np.newaxis]
  print("TRAIN x,y", trainx.shape, trainy.shape)

  mu_x = np.mean(trainx, axis=0)
  sigma_x = np.std(trainx, axis=0)
  trainx = (trainx-mu_x)/sigma_x

  model = get_RNN_model()
  model.fit(trainx, trainy, batch_size=32, epochs=10)

  if unseenid >= 0: #not train with all subj,that is an unseenid available
    testx, testy = np.array(data[unseenid]), np.array(labels[unseenid])
    testy = testy[:, :, np.newaxis]
    testx = (testx-mu_x)/sigma_x
    print("TEST x,y", testx.shape, testy.shape)
    _, accuracy = model.evaluate(testx, testy, batch_size=32)
    print("TESTING ACCURACY with unseenid:", unseenid, accuracy)
    testy_pred = model.predict(testx)
    b = np.squeeze(testy_pred, axis=2)
    a = np.array(labels[unseenid])
    #print("b==>",b.shape, b)
    #print("a==>",a.shape, a)
    print("\n  MAE:", mean_absolute_error(a, b))

  return model

In [ ]:
#left wrist model
for b in subj_dict.items():
  subjid = b[1]
  print("*********Starting left model for unseen ",subjid)
  model = train_model_unseen_id(l_all_data,l_all_labels,subjid)

print("*********Starting left model for ALL ")
model = train_model_unseen_id(l_all_data,l_all_labels,-1)

*********Starting left model for unseen  1
TRAIN x,y (389, 30, 9) (389, 30, 1)
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Epoch 1/10
389/389 [==============================] - 2s 6ms/step - loss: 0.5881 - acc: 0.7342
Epoch 2/10
389/389 [==============================] - 1s 3ms/step - loss: 0.3037 - acc: 0.8941
Epoch 3/10
389/389 [==============================] - 1s 3ms/step - loss: 0.2010 - acc: 0.9154
Epoch 4/10
389/389 [==============================] - 1s 3ms/step - loss: 0.1729 - acc: 0.9224
Epoch 5/10
389/389 [==============================] - 1s 3ms/step - loss: 0.1597 - acc: 0.9308
Epoch 6/10
389/389 [==============================] - 1s 3ms/step - loss: 0.1501 - acc: 0.9376
Epoch 7/10
389/389 [==============================] - 1s 3ms/step - loss: 0.1414 - acc: 0.9429
Epoch 8/10
389/389 [==============================] - 1s 3ms/step - loss: 0.1334 - acc: 0.9445
Epoch 9/10
389/389 [=====================

In [ ]:
#right wrist model
for b in subj_dict.items():
  subjid = b[1]
  print("*********Starting right model for unseen ",subjid)
  model = train_model_unseen_id(r_all_data,r_all_labels,subjid)

print("*********Starting right model for ALL ")
model = train_model_unseen_id(r_all_data,r_all_labels,-1)

In [ ]:
right_model = train_model_unseen_id(r_all_data,r_all_labels,-1)
right_model.save( DATA_PATH +'right_model.h5')

left_model = train_model_unseen_id(l_all_data,l_all_labels,-1)
left_model.save( DATA_PATH +'left_model.h5')

In [ ]:
from keras.models import load_model
import coremltools



coreml_model = coremltools.converters.keras.convert(model, input_names=['meanofsensors'], output_names=['outputarray'])

coreml_model.author = 'SS'
coreml_model.short_description = 'HW Recognition'
coreml_model.save(DATA_PATH + 'mymodel.mlmodel')

0 : bidirectional_1_input, <keras.engine.input_layer.InputLayer object at 0x7f18132f09b0>
1 : bidirectional_1, <keras.layers.wrappers.Bidirectional object at 0x7f18132f0748>
2 : dense_1, <keras.layers.core.Dense object at 0x7f18132e80b8>
3 : dense_1__activation__, <keras.layers.core.Activation object at 0x7f180994df28>


In [ ]:
a = np.array(r_all_data[1])
print(a.shape, a[0])

(68, 30, 9) [[ 1.88175000e-03 -4.09500000e-04 -1.85750000e-04  3.66733000e-01
   6.13450875e-01 -6.99113000e-01  1.55877500e-02 -3.58280000e-02
  -1.09896250e-02]
 [-1.30850000e-03 -1.24840000e-03 -7.39200000e-04  3.48488500e-01
   6.08275700e-01 -7.12914400e-01  1.53205000e-02 -2.10763000e-02
  -8.92690000e-03]
 [-7.44343434e-04 -8.58585859e-04 -4.17575758e-04  3.38369596e-01
   5.96043838e-01 -7.28096364e-01  1.70244444e-02 -1.04121212e-02
   5.05242424e-03]
 [ 3.14534343e-02 -1.72818182e-03  2.52935354e-02  2.99748889e-01
   4.83931111e-01 -7.77799192e-01  5.68006869e-01 -1.81196566e-01
   1.85521212e-02]
 [ 1.19933000e-02  5.02250000e-03  2.12920000e-02  1.87731000e-02
  -8.78887000e-02 -8.07415200e-01  6.81801100e-01 -2.01247800e-01
   2.67322200e-01]
 [-2.51180808e-02 -2.97754545e-02  3.62323232e-04 -2.92308384e-01
  -5.32932626e-01 -7.86119495e-01 -1.31467273e-01 -2.74869697e-01
  -3.75727273e-02]
 [-2.27111111e-03 -1.64329293e-02  2.43695960e-02 -4.01762929e-01
  -3.47645152e-0

In [ ]:
#PRAC
unseenid = 5
data, labels = r_all_data, r_all_labels
trainx, trainy = [],[]
for a in subj_dict.items():
  subjid = a[1]
  if subjid != unseenid:
    trainx.extend(data[subjid])
    trainy.extend(labels[subjid])

trainx, trainy = np.array(trainx), np.array(trainy)
trainy = trainy[:, :, np.newaxis]
print("TRAIN x,y", trainx.shape, trainy.shape)

mu_x = np.mean(trainx, axis=0)
sigma_x = np.std(trainx, axis=0)
trainx = (trainx-mu_x)/sigma_x

model = get_RNN_model()
model.fit(trainx, trainy, batch_size=32, epochs=2)

if unseenid >= 0: #not train with all subj,that is an unseenid available
  testx, testy = np.array(data[unseenid]), np.array(labels[unseenid])
  testy = testy[:, :, np.newaxis]
  testx = (testx-mu_x)/sigma_x
  print("TEST x,y", testx.shape, testy.shape)
  _, accuracy = model.evaluate(testx, testy, batch_size=32)
  print("TESTING ACCURACY with unseenid:", unseenid, accuracy)
  testy_pred = model.predict(testx)
  b = np.squeeze(testy_pred, axis=2)
  a = np.array(labels[unseenid])
  print("b==>",b.shape, b)
  print("a==>",a.shape, a)
  print("\n  MAE:", mean_absolute_error(a, b))

TRAIN x,y (214, 30, 9) (214, 30, 1)
Epoch 1/2
7/7 [==============================] - 1s 87ms/step - loss: 0.6419 - accuracy: 0.6611
Epoch 2/2
7/7 [==============================] - 1s 84ms/step - loss: 0.5247 - accuracy: 0.7988
TEST x,y (246, 30, 9) (246, 30, 1)
8/8 [==============================] - 0s 12ms/step - loss: 0.2857 - accuracy: 0.9466
TESTING ACCURACY with unseenid: 5 0.9466124773025513
b==> (246, 30) [[0.18191826 0.17178833 0.16548488 ... 0.21646753 0.28981584 0.34182948]
 [0.35072333 0.31805506 0.26409507 ... 0.17849317 0.19075885 0.25719333]
 [0.4080287  0.3820942  0.3833681  ... 0.18208554 0.19578779 0.26048723]
 ...
 [0.11093304 0.09558859 0.0832777  ... 0.11037618 0.12687504 0.14208916]
 [0.10780212 0.09330711 0.08171228 ... 0.10862166 0.1245625  0.14115855]
 [0.11208293 0.09665805 0.08422342 ... 0.13503307 0.15787236 0.16430743]]
a==> (246, 30) [[0 0 0 ... 0 0 1]
 [1 1 1 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]

  M

In [ ]:
#PRAC
b[2]

array([0.4080287 , 0.3820942 , 0.3833681 , 0.42438504, 0.45861602,
       0.45025596, 0.43466216, 0.4126091 , 0.3859394 , 0.3559332 ,
       0.31626064, 0.27682197, 0.226796  , 0.18205822, 0.202808  ,
       0.17111894, 0.1433286 , 0.18324238, 0.20787677, 0.25436586,
       0.31577864, 0.300775  , 0.26714414, 0.24058115, 0.22277051,
       0.20816603, 0.19575971, 0.18208554, 0.19578779, 0.26048723],
      dtype=float32)

In [ ]:
a[2]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1,
       1, 1, 1, 1, 1, 0, 0, 0])

In [ ]:
y_actual = np.array(r_all_labels[5])
y_pred = md.predict(y_actual)


ValueError: ignored

In [ ]:
a = np.squeeze(y_pred, axis=2).shape

In [ ]:
a.shape

AttributeError: ignored

In [ ]:
mean_absolute_error(y_pred, b)

In [ ]:
np.array(r_all_labels[5]).shape

(246, 30)

In [ ]:
lmain = []
list1 = [1,2,3]
list2 = [4,5,6]

lmain.append(list1)
lmain

[[1, 2, 3]]

In [ ]:
lmain.append(list2)
lmain

[[1, 2, 3], [4, 5, 6]]

In [ ]:
lmain.extend(list2)
lmain

[[1, 2, 3], [4, 5, 6], 4, 5, 6]

In [ ]:
tmp_data_list = []
res = [[81,82,84,85,89,1],[91,92,94,95,99,1]]

#for j in range(i, (i+bunch_size), 1):

tmp_data_list.append(res[0][:-1])
tmp_data_list

[[81, 82, 84, 85, 89]]

In [ ]:
tmp_data_list.append(res[1][:-1])
tmp_data_list

[[81, 82, 84, 85, 89], [91, 92, 94, 95, 99]]

In [ ]:
lmain = []
list1 = [[[1,9],[20,29],[30,39]],[[101,109],[201,209],[301,309]]]

list2 = [[[7,9],[80,89],[90,99]],[[701,709],[801,809],[901,909]]]

In [ ]:
lmain.append(list1)
lmain.append(list2)
lmain

[[[[1, 9], [20, 29], [30, 39]], [[101, 109], [201, 209], [301, 309]]],
 [[[7, 9], [80, 89], [90, 99]], [[701, 709], [801, 809], [901, 909]]]]

In [ ]:
lmain.extend(list1)
lmain.extend(list2)
lmain

[[[[1, 9], [20, 29], [30, 39]], [[101, 109], [201, 209], [301, 309]]],
 [[[7, 9], [80, 89], [90, 99]], [[701, 709], [801, 809], [901, 909]]],
 [[1, 9], [20, 29], [30, 39]],
 [[101, 109], [201, 209], [301, 309]],
 [[7, 9], [80, 89], [90, 99]],
 [[701, 709], [801, 809], [901, 909]]]